In [3]:
from FINData import FINData
import vectorbtpro as vbt
data=FINData.pull(
    ['HPG','VNM'],
    start='2014-01-01'
)

In [4]:
data.get("Close", "HPG").head()

DateTime
2014-01-02 00:00:00+00:00    2.26
2014-01-03 00:00:00+00:00    2.29
2014-01-06 00:00:00+00:00    2.32
2014-01-07 00:00:00+00:00    2.32
2014-01-08 00:00:00+00:00    2.34
Name: Close, dtype: float64

In [5]:
close = data.get("Close")
open_ = data.get("Open")

signal = close > open_
signal["HPG"].head()

DateTime
2014-01-02 00:00:00+00:00    False
2014-01-03 00:00:00+00:00     True
2014-01-06 00:00:00+00:00     True
2014-01-07 00:00:00+00:00    False
2014-01-08 00:00:00+00:00     True
Name: HPG, dtype: bool

In [6]:
ret = data.get("Close").pct_change()

signal_up3 = ret > 0.03
signal_up3["HPG"]

DateTime
2014-01-02 00:00:00+00:00    False
2014-01-03 00:00:00+00:00    False
2014-01-06 00:00:00+00:00    False
2014-01-07 00:00:00+00:00    False
2014-01-08 00:00:00+00:00    False
                             ...  
2025-11-19 00:00:00+00:00    False
2025-11-20 00:00:00+00:00    False
2025-11-21 00:00:00+00:00    False
2025-11-24 00:00:00+00:00    False
2025-11-25 00:00:00+00:00    False
Name: HPG, Length: 2967, dtype: bool

In [7]:
signal_down2 = ret < -0.02
signal_down2

symbol,HPG,VNM
DateTime,,
2014-01-02 00:00:00+00:00,False,False
2014-01-03 00:00:00+00:00,False,False
2014-01-06 00:00:00+00:00,False,False
2014-01-07 00:00:00+00:00,False,False
2014-01-08 00:00:00+00:00,False,False
...,...,...
2025-11-19 00:00:00+00:00,False,False
2025-11-20 00:00:00+00:00,False,False
2025-11-21 00:00:00+00:00,False,False


In [11]:
fast = vbt.MA.run(data.get("Close"), 10)
slow = vbt.MA.run(data.get("Close"), 20)

entries_ma = fast.vbt.crossed_above(slow.ma)
exits = fast.vbt.crossed_below(slow.ma)

In [12]:
rsi=vbt.talib('RSI').run(
    data.get("Close")
)

In [13]:

entries = (
    fast.vbt.crossed_below(slow.ma)
    & (rsi.rsi < 30)
)

In [14]:
entries.sum()

ma_window  ma_window  symbol
10         20         HPG       2
                      VNM       3
dtype: int64

In [16]:
bb_mult=vbt.talib("BBANDS").run(
    data.get("Close"),
    timeperiod=vbt.Default(14),
    nbdevup=[2,3],
    nbdevdn=[2,3]
)
data.get("Low")<bb_mult.lowerband


ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [18]:
mask=data.get("Low").vbt<bb_mult.lowerband

In [19]:
mask.sum()

bbands_nbdevup  bbands_nbdevdn  symbol
2               2               HPG       316
                                VNM       327
3               3               HPG        39
                                VNM        44
dtype: int64

In [23]:
bb=vbt.talib("BBANDS").run(
    data.get("Close"),
    timeperiod=vbt.Default(14),
    nbdevup=vbt.Default(2),
    nbdevdn=vbt.Default(2)
)
bb.lowerband

symbol,HPG,VNM
DateTime,,
2014-01-02 00:00:00+00:00,NaN,NaN
2014-01-03 00:00:00+00:00,NaN,NaN
2014-01-06 00:00:00+00:00,NaN,NaN
2014-01-07 00:00:00+00:00,NaN,NaN
2014-01-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-11-19 00:00:00+00:00,25.717624,55.354120
2025-11-20 00:00:00+00:00,25.717565,55.628393
2025-11-21 00:00:00+00:00,25.839158,56.008477


In [25]:
bandwidth=(bb.upperband-bb.lowerband)/bb.middleband 
mask=bandwidth.vbt>vbt.Param([0.15,0.3],name='threadsold')
mask.sum()

threadsold  symbol
0.15        HPG       610
            VNM       168
0.30        HPG        59
            VNM         0
dtype: int64

In [28]:
import pandas as pd 
mask = pd.concat(
    (bandwidth > 0.15, bandwidth > 0.3), 
    keys=pd.Index([0.15, 0.3], name="threshold"), 
    axis=1
)
mask.sum()

threshold  symbol
0.15       HPG       610
           VNM       168
0.30       HPG        59
           VNM         0
dtype: int64

In [34]:
low_below_lband=data.get("Low")<bb.lowerband
mask=low_below_lband.vbt.signals.first(after_false=True)
mask.sum()

symbol
HPG    131
VNM    148
dtype: int64

In [ ]:
low_below_lband.sum()

symbol
HPG    316
VNM    327
dtype: int64

In [33]:
btc_low = data.get("Low", "HPG").rename("Low")  
btc_lowerband = bb.lowerband["HPG"].rename("Lower Band")
btc_mask = mask["HPG"].rename("Signals")

fig = btc_low.vbt.plot()  
btc_lowerband.vbt.plot(fig=fig)
btc_mask.vbt.signals.plot_as_markers(
    y=btc_low, 
    trace_kwargs=dict(
        marker=dict(
            color="#DFFF00"
        )
    ),
    fig=fig
)  
fig.show()

In [39]:
import numpy as np
sample_low = pd.Series([10, 9, 8, 9, 8])
sample_lband = pd.Series([np.nan, np.nan, 9, 8, 9])
sample_mask = sample_low < sample_lband
sample_lband

0    NaN
1    NaN
2    9.0
3    8.0
4    9.0
dtype: float64

In [40]:
sample_lband.ffill().isnull()

0     True
1     True
2    False
3    False
4    False
dtype: bool

In [ ]:
buffer = sample_lband.ffill().isnull().sum(axis=0).max()  
buffer


sample_buf_mask = sample_low.iloc[buffer:] < sample_lband.iloc[buffer:]
sample_buf_mask = sample_buf_mask.vbt.signals.first(after_false=True)
sample_mask = sample_low.vbt.wrapper.fill(False)
sample_mask.loc[sample_buf_mask.index] = sample_buf_mask

In [42]:
sample_buf_mask

2    False
3    False
4     True
dtype: bool

In [ ]:
vbt.IF.Indicator

In [ ]:
sample_mask

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [48]:
close = data.get("Close")
wrapper = close.vbt.wrapper

print(wrapper.index)
print(wrapper.columns)
print(wrapper.shape)

DatetimeIndex(['2014-01-02 00:00:00+00:00', '2014-01-03 00:00:00+00:00',
               '2014-01-06 00:00:00+00:00', '2014-01-07 00:00:00+00:00',
               '2014-01-08 00:00:00+00:00', '2014-01-09 00:00:00+00:00',
               '2014-01-10 00:00:00+00:00', '2014-01-13 00:00:00+00:00',
               '2014-01-14 00:00:00+00:00', '2014-01-15 00:00:00+00:00',
               ...
               '2025-11-12 00:00:00+00:00', '2025-11-13 00:00:00+00:00',
               '2025-11-14 00:00:00+00:00', '2025-11-17 00:00:00+00:00',
               '2025-11-18 00:00:00+00:00', '2025-11-19 00:00:00+00:00',
               '2025-11-20 00:00:00+00:00', '2025-11-21 00:00:00+00:00',
               '2025-11-24 00:00:00+00:00', '2025-11-25 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='DateTime', length=2967, freq=None)
Index(['HPG', 'VNM'], dtype='object', name='symbol')
(2967, 2)


In [54]:
anchor_points = data.wrapper.get_index_points(  
    every="M", 
    start=0,  
    exact_start=True
)
anchor_points

array([   0,   18,   35,   56,   76,   96,  117,  140,  161,  181,  204,
        224,  247,  267,  281,  303,  322,  342,  364,  387,  408,  429,
        451,  472,  495,  515,  531,  554,  574,  594,  616,  637,  660,
        681,  702,  724,  746,  763,  782,  805,  824,  845,  867,  888,
        911,  931,  953,  975,  996, 1016, 1031, 1053, 1072, 1094, 1115,
       1137, 1160, 1179, 1202, 1224, 1244, 1266, 1281, 1302, 1321, 1343,
       1363, 1386, 1408, 1428, 1451, 1472, 1494, 1511, 1531, 1553, 1573,
       1593, 1615, 1638, 1659, 1680, 1702, 1723, 1746, 1766, 1781, 1804,
       1824, 1844, 1866, 1888, 1910, 1930, 1951, 1973, 1996, 2015, 2031,
       2054, 2074, 2094, 2116, 2137, 2160, 2180, 2201, 2223, 2245, 2261,
       2281, 2304, 2324, 2344, 2366, 2387, 2410, 2429, 2451, 2473, 2494,
       2516, 2532, 2553, 2572, 2594, 2614, 2637, 2659, 2678, 2701, 2722,
       2744, 2761, 2781, 2802, 2822, 2842, 2863, 2886, 2907, 2927, 2950],
      dtype=int64)

In [55]:

left_bound = np.full(len(data.wrapper.index), np.nan) 
left_bound[anchor_points] = anchor_points
left_bound

array([ 0., nan, nan, ..., nan, nan, nan])

In [59]:
from numpy import int_
left_bound = vbt.nb.ffill_1d_nb(left_bound).astype(int_)

In [60]:
left_bound

array([   0,    0,    0, ..., 2950, 2950, 2950])

In [62]:
left_bound = bandwidth.index[left_bound]
left_bound

DatetimeIndex(['2014-01-02 00:00:00+00:00', '2014-01-02 00:00:00+00:00',
               '2014-01-02 00:00:00+00:00', '2014-01-02 00:00:00+00:00',
               '2014-01-02 00:00:00+00:00', '2014-01-02 00:00:00+00:00',
               '2014-01-02 00:00:00+00:00', '2014-01-02 00:00:00+00:00',
               '2014-01-02 00:00:00+00:00', '2014-01-02 00:00:00+00:00',
               ...
               '2025-11-03 00:00:00+00:00', '2025-11-03 00:00:00+00:00',
               '2025-11-03 00:00:00+00:00', '2025-11-03 00:00:00+00:00',
               '2025-11-03 00:00:00+00:00', '2025-11-03 00:00:00+00:00',
               '2025-11-03 00:00:00+00:00', '2025-11-03 00:00:00+00:00',
               '2025-11-03 00:00:00+00:00', '2025-11-03 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='DateTime', length=2967, freq=None)

In [63]:
right_bound = data.wrapper.index  
right_bound

DatetimeIndex(['2014-01-02 00:00:00+00:00', '2014-01-03 00:00:00+00:00',
               '2014-01-06 00:00:00+00:00', '2014-01-07 00:00:00+00:00',
               '2014-01-08 00:00:00+00:00', '2014-01-09 00:00:00+00:00',
               '2014-01-10 00:00:00+00:00', '2014-01-13 00:00:00+00:00',
               '2014-01-14 00:00:00+00:00', '2014-01-15 00:00:00+00:00',
               ...
               '2025-11-12 00:00:00+00:00', '2025-11-13 00:00:00+00:00',
               '2025-11-14 00:00:00+00:00', '2025-11-17 00:00:00+00:00',
               '2025-11-18 00:00:00+00:00', '2025-11-19 00:00:00+00:00',
               '2025-11-20 00:00:00+00:00', '2025-11-21 00:00:00+00:00',
               '2025-11-24 00:00:00+00:00', '2025-11-25 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='DateTime', length=2967, freq=None)

In [ ]:
mask = (bandwidth <= 0.1).vbt.resample_between_bounds(  
    left_bound, 
    right_bound,
    "any",
    closed_lbound=True,  
    closed_rbound=True,
    wrap_kwargs=dict(fillna=0, dtype=bool)
)
mask.index = right_bound